# Library

In [1]:
import os
import pandas as pd
import numpy as np

from datasets import Dataset
from dotenv import load_dotenv
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, pipeline

# Labeling

In [5]:
data_path = r"C:\Users\james\J_Data_Lab\Project-FXCast\crawler\data\investing_comments_cleaned_final.csv"
df = pd.read_csv(data_path)

# summary 컬럼만 추출 + 중복 제거, 공백 제거
df_labeled = df[["content"]].dropna().drop_duplicates()
df_labeled["content"] = df_labeled["content"].str.strip()

# 감정 라벨 컬럼
df_labeled["label"] = ""

# 저장
save_path = r"C:\Users\james\J_Data_Lab\Project-FXCast\crawler\data\investing_comments_final_labeling.csv"
df_labeled.to_csv(save_path, index=False, encoding="utf-8-sig")
print(f"{len(df_labeled)}개 문장 저장 완료: {save_path}")

279466개 문장 저장 완료: C:\Users\james\J_Data_Lab\Project-FXCast\crawler\data\investing_comments_final_labeling.csv


# Fine Tuning

* 환율 댓글 튜닝
* 0 : 공포
* 1 : 탐욕

In [24]:
data_path = r"C:\Users\james\J_Data_Lab\Project-FXCast\crawler\data\investing_comments_final_labeling_v3.csv"
df = pd.read_csv(data_path)
# 라벨 정리
df["label"] = df["label"].astype(str).str.strip()
label_map = {"0": 0, "1": 1} # 0은 공포 1은 욕심
df["label"] = df["label"].map(label_map)
df_clean = df[df["label"].isin([0, 1])].copy()
df_clean["label"] = df_clean["label"].astype(int)
dataset = Dataset.from_pandas(df[["content", "label"]])

In [26]:
# https://huggingface.co/snunlp/KR-FinBert-SC 
model_name = "snunlp/KR-FinBERT-SC"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2, ignore_mismatched_sizes=True)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at snunlp/KR-FinBERT-SC and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
def tokenize(batch):
    return tokenizer(batch["content"], padding="max_length", truncation=True, max_length=512)

dataset = dataset.map(tokenize, batched=True)
dataset = dataset.train_test_split(test_size=0.1)

Map:   0%|          | 0/237 [00:00<?, ? examples/s]

In [28]:
training_args = TrainingArguments(
    output_dir="./finbert-finetuned",
    per_device_train_batch_size=8,
    num_train_epochs=4,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
)

trainer.train()

C:\Users\james\anaconda3\envs\my_new_env\lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.582000,0.492887
2,0.178400,0.756978
3,0.158800,0.707856
4,0.008000,0.760682


TrainOutput(global_step=108, training_loss=0.2580258820067953, metrics={'train_runtime': 22.8621, 'train_samples_per_second': 37.267, 'train_steps_per_second': 4.724, 'total_flos': 56042654791680.0, 'train_loss': 0.2580258820067953, 'epoch': 4.0})

In [29]:
trainer.save_model("./finbert-finetuned")

# test
pipe = pipeline("text-classification", model="./finbert-finetuned", tokenizer=tokenizer)
pipe("심상치 않네요 중동전쟁 발발하고 1월8일에 김정은이 미사일 실험하고 그러면 1170은 기본이고 1190원도 순식간일듯.")

Device set to use cuda:0


[{'label': 'LABEL_1', 'score': 0.7681488990783691}]

In [30]:
pipe = pipeline("text-classification", model="./finbert-finetuned", tokenizer=tokenizer)
pipe("매수잡아보자ㅎ")

Device set to use cuda:0


[{'label': 'LABEL_1', 'score': 0.8322440385818481}]

In [31]:
pipe = pipeline("text-classification", model="./finbert-finetuned", tokenizer=tokenizer)
pipe("내가 작년여름부터 장기하방이라했자나")

Device set to use cuda:0


[{'label': 'LABEL_0', 'score': 0.9057745337486267}]

In [32]:
# hugging face upload
load_dotenv(dotenv_path=".env")
token = os.getenv("HF_TOKEN")

model.push_to_hub("DataWizardd/finbert-sentiment-krw-comment-v3", token=token)
tokenizer.push_to_hub("DataWizardd/finbert-sentiment-krw-comment-v3", token=token)

model.safetensors:   0%|          | 0.00/406M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/DataWizardd/finbert-sentiment-krw-comment-v3/commit/90f46f18696e9ca4257e66b8805d7261fca66c8a', commit_message='Upload tokenizer', commit_description='', oid='90f46f18696e9ca4257e66b8805d7261fca66c8a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/DataWizardd/finbert-sentiment-krw-comment-v3', endpoint='https://huggingface.co', repo_type='model', repo_id='DataWizardd/finbert-sentiment-krw-comment-v3'), pr_revision=None, pr_num=None)

# Classification Report

In [33]:
data_path = r"C:\Users\james\J_Data_Lab\Project-FXCast\crawler\data\investing_comments_final_labeling_v3.csv"
df = pd.read_csv(data_path)

# 라벨 매핑
df["label"] = df["label"].astype(str).str.strip()
label_map = {"0": 0, "1": 1}
df["label"] = df["label"].map(label_map)

df_clean = df[df["label"].isin([0, 1])].copy()
df_clean["label"] = df_clean["label"].astype(int)

In [34]:
train_df, test_df = train_test_split(
    df_clean,
    test_size=0.2,
    stratify=df_clean["label"],
    random_state=42
)

In [35]:
# 데이터셋 변환
train_dataset = Dataset.from_pandas(train_df.reset_index(drop=True))
test_dataset = Dataset.from_pandas(test_df.reset_index(drop=True))

# 토크나이저 로드 (KR-FinBERT-SC)
tokenizer = AutoTokenizer.from_pretrained("snunlp/KR-FinBERT-SC")

# 토크나이징 함수
def tokenize(batch):
    return tokenizer(batch["content"], padding="max_length", truncation=True, max_length=128)

train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

# 필요한 컬럼만 학습용으로 설정
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

Map:   0%|          | 0/189 [00:00<?, ? examples/s]

Map:   0%|          | 0/48 [00:00<?, ? examples/s]

In [36]:
# 예측
preds = trainer.predict(test_dataset)

# 결과 분석
y_true = preds.label_ids
y_pred = np.argmax(preds.predictions, axis=1)

print(classification_report(y_true, y_pred, target_names=["0", "1"]))

              precision    recall  f1-score   support

           0       0.95      0.91      0.93        22
           1       0.93      0.96      0.94        26

    accuracy                           0.94        48
   macro avg       0.94      0.94      0.94        48
weighted avg       0.94      0.94      0.94        48

